In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine.reflection import Inspector
from tqdm import tqdm

In [2]:
building_ids_df = pd.read_excel("../inputs/buliding_ids_for_data.xlsx")
building_ids = tuple(building_ids_df.iloc[:, 0].tolist())

In [3]:
db_user = "nycdb"
db_password = "nycdb"
db_host = "127.0.0.1"
db_port = "5432"
db_name = "nycdb"

engine = create_engine(
    f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)

inspector = Inspector.from_engine(engine)

# Get table information
print("Table Names:")
print("\n".join(inspector.get_table_names()))

/var/folders/yl/x_8rvst10b3684hd_346t7cr0000gn/T/ipykernel_82770/715051296.py:11: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


Table Names:
spatial_ref_sys
real_property_master
real_property_parties
real_property_references
real_property_remarks
real_property_legals
personal_property_master
personal_property_parties
personal_property_references
personal_property_remarks
acris_country_codes
acris_document_control_codes
acris_property_type_codes
acris_ucc_collateral_codes
personal_property_legals
dob_complaints
dob_safety_violations
dob_violations
hpd_complaints_and_problems
hpd_violations
topology
layer
featnames
geocode_settings
geocode_settings_default
direction_lookup
secondary_unit_lookup
state_lookup
street_type_lookup
place_lookup
county_lookup
countysub_lookup
zip_lookup_all
zip_lookup_base
zip_lookup
county
state
place
zip_state
zip_state_loc
cousub
edges
addrfeat
addr
zcta5
tabblock20
faces
loader_platform
loader_variables
loader_lookuptables
tract
tabblock
bg
pagc_gaz
pagc_lex
pagc_rules


In [4]:
TABLES = [
    "dob_complaints",
    "dob_safety_violations",
    "dob_violations",
    "hpd_complaints_and_problems",
    "hpd_violations",
]

for table in tqdm(TABLES):
    query = f"""
        SELECT * FROM {table}
        WHERE CAST(bin AS TEXT) IN ({', '.join(f"'{id}'" for id in building_ids)}) 
        """
    pd.read_sql(query, engine).to_csv(
        f"../csv_samples/{table}_on_buildingid.csv",
        index=False,
    )

100%|██████████| 5/5 [00:13<00:00,  2.62s/it]
